In [3]:
from pandas import *
from datetime import timedelta
from io import open
import json
import pandas as pd

    

# def load_save_materials(filename):
#     with open(filename) as json_file:
#     diccionario = json.load(json_file)
    
        
def importMaterials(filename):
    br = open(filename, "r")
    osm = br.readlines()
    i = 0
    materials = []
    materiales = {}   # este es el nuevo diccionario que ir'a creciendo
#     osm_sin = open
    while (i < len(osm)):
        if("OS:Material," in osm[i]):
            i, material,nuevo = getOsmObject(osm, i)  # ahora regresa tres argumentos, el tercero el dic
            materiales.update(nuevo)  #actualiza el diccionario materiales con el nuevo encontrado
#             print(nuevo)
            materials.append(material)
        i += 1
    br.close()
    materials = pd.DataFrame(materials)
    materials.index = materials["Name"]
    return materials,materiales   # regresa ahora el diccionario materiales

def getOsmObject(list, index):
    object = {}
    index += 1
    while list[index] != "\n":
        if (',' in list[index]):
            sep = ','
        else:
            sep = ';'
        name = list[index].split(sep)[1]
        name = name.split('!')[1][2:]
        name = name.split('\n')[0]

        object[name] = getValue(list[index].split(sep)[0][2:])
        index += 1
    nombre = object["Name"]  #extrae el nombre del material para definir diccionario
    nuevo_dic = {nombre:object}  # crea nuevo diccionario con el nombre del material
    return index, object,nuevo_dic  # regresa nuevo diccionario

def getValue(strvalue):
    try:
        value = float(strvalue)
        if ("." not in strvalue):
            value = int(value)
    except ValueError:
        value = strvalue
    return value

def adjustTimestamp(strDate, year="2020"):
    strDate = year + "/" + strDate.strip()
    wDate = strDate.split()[0]
    wTime = strDate.split()[1]
    tsDate = to_datetime(wDate, format="%Y/%m/%d")
    wHour = int(wTime.split(":")[0])
    wMin = int(wTime.split(":")[1])
    if (wHour > 23):
        wHour = 0
        tsDate += timedelta(days=1)
    tsDate = tsDate.replace(hour=wHour, minute=wMin)
    return tsDate

def getEPVariables(filename):
    datainfo = read_csv(filename)
    datainfo["Date/Time"] = datainfo["Date/Time"].apply(lambda x: adjustTimestamp(x))
    datainfo.rename(columns={"Date/Time":"Timestamp"}, inplace=True)
    datainfo.set_index("Timestamp",drop=True,inplace=True)
    return datainfo

def getValue(strvalue):
    try:
        value = float(strvalue)
        if ("." not in strvalue):
            value = int(value)
    except ValueError:
        value = strvalue
    return value

 
def setOsmObject(list, index, object):
    index += 1
    for name, value in object.items():
        if str(value)=="nan":
          break
        if (',' in list[index]):
            sep = ','
        else:
            sep = ';'
        list[index] = "  " + str(value) + sep + "!- " + name + "\n"
        index += 1
    return True


In [50]:
m1, m2 = importMaterials("../osm/edificio_este.osm")


In [57]:

def exportMaterials(filename, materials,materiales):
    br = open(filename, "r")
    osm = br.readlines()
    br.close()
    i = 0
    while (i < len(osm)):
        if("OS:Material," in osm[i]):
            material = materials.loc[osm[i+2].split(',')[0][2:]]
#             print("OSM",osm[i])
            print("MaterialesAAA",materials.loc[osm[i+2].split(',')[0][2:]])
            print("BBB",materiales[osm[i+2].split(',')[0][2:]])
            setOsmObject(osm, i, material)
        i += 1
    br = open(filename, "w")
    for line in osm:
        br.write(line)
    br.close()
    return True
exportMaterials("../osm/edificio_este.osm",m1,m2)

MaterialesAAA Handle                    {2e7157dc-32c2-44e3-9415-7874ececbba9}
Name                              concreto_fc150_granzon_L8_a0pX
Roughness                                            MediumRough
Thickness {m}                                               0.15
Conductivity {W/m-K}                                         1.3
Density {kg/m3}                                           1800.0
Specific Heat {J/kg-K}                                    1000.0
Thermal Absorptance                                          0.8
Solar Absorptance                                            0.3
Visible Absorptance                                          1.0
Name: concreto_fc150_granzon_L8_a0pX, dtype: object
BBB {'Handle': '{2e7157dc-32c2-44e3-9415-7874ececbba9}', 'Name': 'concreto_fc150_granzon_L8_a0pX', 'Roughness': 'MediumRough', 'Thickness {m}': 0.15, 'Conductivity {W/m-K}': 1.3, 'Density {kg/m3}': 1800.0, 'Specific Heat {J/kg-K}': 1000.0, 'Thermal Absorptance': 0.8, 'Solar Absorptan

True

In [47]:
exportMaterials("../osm/edificio_este.osm",m1)

Materiales Handle                    {2e7157dc-32c2-44e3-9415-7874ececbba9}
Name                              concreto_fc150_granzon_L8_a0pX
Roughness                                            MediumRough
Thickness {m}                                               0.15
Conductivity {W/m-K}                                         1.3
Density {kg/m3}                                           1800.0
Specific Heat {J/kg-K}                                    1000.0
Thermal Absorptance                                          0.8
Solar Absorptance                                            0.3
Visible Absorptance                                          1.0
Name: concreto_fc150_granzon_L8_a0pX, dtype: object
Materiales Handle                    {acc9b317-088e-4fa3-a849-9aec59e11271}
Name                                      concreto_fc150_L6_a0pX
Roughness                                            MediumRough
Thickness {m}                                               0.15
Conductivity {W/

True

In [28]:
m1.loc['concreto_fc150_granzon_L8_a0pX',"Visible Absorptance"] = 1.0